In [9]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

os.chdir('/content/drive/My Drive/data/dream_at_scale')

### 8.2.1　用Keras 实现DeepDream
我们将从一个在ImageNet 上预训练的卷积神经网络开始。Keras 中有许多这样的卷积神经
网络：VGG16、VGG19、Xception、ResNet50 等。我们可以用其中任何一个来实现DeepDream，
但我们选择的卷积神经网络会影响可视化的效果，因为不同的卷积神经网络架构会学到不同的
特征。最初发布的DeepDream 中使用的卷积神经网络是一个Inception 模型，在实践中，人们已
经知道Inception 能够生成漂亮的DeepDream 图像，所以我们将使用Keras 内置的Inception V3
模型。

-  　加载预训练的Inception V3 模型

In [0]:
from keras.applications import inception_v3
from keras import backend as K

In [2]:
K.set_learning_phase(0)#我们不需要训练模型，所以这个命令会禁用所有与训练有关的操作

#构建不包括全连接层的Inception V3网络。使用预训练的ImageNet 权重来加载模型
model = inception_v3.InceptionV3(weights='imagenet',
                                include_top=False)

W0815 05:17:35.385249 139756634847104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 05:17:35.427502 139756634847104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 05:17:35.434779 139756634847104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 05:17:35.466443 139756634847104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0815 05:17:35.467444 139756634

87916544/87910968 [==============================] - 6s 0us/step


接下来，我们要计算损失（loss），即在梯度上升过程中需要最大化的量。在第5 章的过滤
器可视化中，我们试图将某一层的某个过滤器的值最大化。这里，我们要将多个层的所有过滤
器的激活同时最大化。具体来说，就是对一组靠近顶部的层激活的L2 范数进行加权求和，然
后将其最大化。选择哪些层（以及它们对最终损失的贡献）对生成的可视化结果具有很大影响，
所以我们希望让这些参数变得易于配置。更靠近底部的层生成的是几何图案，而更靠近顶部的
层生成的则是从中能够看出某些ImageNet 类别（比如鸟或狗）的图案。我们将随意选择4 层的
配置，但你以后一定要探索多个不同的配置。

- 设置DeepDream 配置

In [0]:
#这个字典将层的名称映射为一个系数，这个系数定量
#表示该层激活对你要最大化的损失的贡献大小。注意，
#层的名称硬编码在内置的Inception V3 应用中。可以
#使用model.summary() 列出所有层的名称

layer_contributions = {'mixed2':0.2,
                      'mixed3':3.,
                      'mixed4':2.,
                      'mixed5':1.5}

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

接下来，我们来定义一个包含损失的张量，损失就是代码清单8-9 中层激活的L2 范数的加
权求和。

- 定义需要最大化的损失

In [5]:
#创建一个字典，将层的名称映射为层的实例
layer_dict = dict([(layer.name,layer) for layer in model.layers])

loss = K.variable(0.)#在定义损失时将层的贡献添加到这个标量变量中
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output #获取层的输出
  
  scaling = K.prod(K.cast(K.shape(activation),'float32'))
  #将该层特征的L2 范数添加到loss中。为了避免出现边界伪影，损失中仅包含非边界的像素
  loss += coeff * K.sum(K.square(activation[: , 2: -2, 2: -2, :])) / scaling 

W0815 06:30:26.514876 139756634847104 variables.py:2429] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


下面来设置梯度上升过程。

- 　梯度上升过程

In [0]:
dream = model.input #这个张量用于保存生成的图像，即梦境图像

grads = K.gradients(loss, dream)[0] #计算损失相对于梦境图像的梯度

grads /= K.maximum(K.mean(K.abs(grads)),1e-7)#将梯度标准化（重要技巧）


#给定一张输出图像，设置一个Keras 函数来获取损失值和梯度值
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs) 

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values =outs[1]
  return loss_value, grad_values


#这个函数运行iterations次梯度上升
def gradient_ascent(x, iterations, step, max_loss=None):
  for i in range(iterations):
    loss_value, grad_values = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    
    print('...Loss value at', i, ':', loss_value)
    x += step * grad_values
  return x
     

最后就是实际的DeepDream 算法。首先，我们来定义一个列表，里面包含的是处理图像的
尺度（也叫八度）。每个连续的尺度都是前一个的1.4 倍（放大40%），即首先处理小图像，然
后逐渐增大图像尺寸（见图8-4）

对于每个连续的尺度，从最小到最大，我们都需要在当前尺度运行梯度上升，以便将之前
定义的损失最大化。每次运行完梯度上升之后，将得到的图像放大40%。


在每次连续的放大之后（图像会变得模糊或像素化），为避免丢失大量图像细节，我们可
以使用一个简单的技巧：每次放大之后，将丢失的细节重新注入到图像中。这种方法是可行的，
因为我们知道原始图像放大到这个尺寸应该是什么样子。给定一个较小的图像尺寸S 和一个较
大的图像尺寸L，你可以计算将原始图像大小调整为L 与将原始图像大小调整为S 之间的区别，
这个区别可以定量描述从S 到L 的细节损失。

- 在多个连续尺度上运行梯度上升

In [29]:
!pip install Pillow

In [0]:
#辅助函数
import scipy
#import scipy.misc
from keras.preprocessing import image
import imageio

def resize_img(img, size):
  img = np.copy(img)
  factors = (1,
            float(size[0])/ img.shape[1],
            float(size[1])/ img.shape[2],
            1)
  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  pil_img = deprocess_image(np.copy(img))
  imageio.imwrite(fname, pil_img)
  #scipy.misc.imsave(fname,pil_img)
  
#通用函数，用于打开图像、改变图像大小以及将图像格式转换为Inception V3 模型能够处理的张量
def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img,axis=0)
  img = inception_v3.preprocess_input(img)
  return img

#通用函数，将一个张量转换为有效图像
def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1,2,0))
    
  else:
    #对inception_v3.preprocess_input所做的预处理进行反向操作
    x = x.reshape((x.shape[1],x.shape[2],3)) 
    
  x /=2.
  x +=0.5
  x *= 255.
  x = np.clip(x,0,255).astype('uint8')
  return x

In [0]:
import numpy as np

In [43]:
#梯度上升的步长
step = 0.01 #@param {type:"number"}
#运行梯度上升的尺度个数
num_octave = 3 #@param {type:"integer"}
#两个尺度之间的大小比例
octave_scale = 1.3 #@param {type:"number"}
#在每个尺度上运行梯度上升的步数
iterations = 10 #@param {type:"number"}

#如果损失增大到大于10，我们要中断梯度上升过程，以避免得到丑陋的伪影
max_loss = 15. #@param {type:"number"}

#将这个变量修改为你要使用的图像的路径
base_image_path = '/content/drive/My Drive/data/dogs-vs-cats/train/cat.37.jpg' #@param {type: "string"}


img = preprocess_image(base_image_path)#将基础图像加载成一个Numpy 数组

original_shape = img.shape[1:3]

#准备一个由形状元组组成的列表，它定义了运行梯度上升的不同尺度
successive_shapes = [original_shape]
for i in range(1, num_octave):
  shape = tuple([int(dim /(octave_scale ** i)) for dim in original_shape])
  successive_shapes.append(shape)
  

successive_shapes = successive_shapes[::-1]#将形状列表反转，变为升序


#将图像Numpy 数组的大小缩放到最小尺寸
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])


for shape in successive_shapes:
  print('Processing image shape',shape)
  img = resize_img(img,shape) #将梦境图像放大
  
  #运行梯度上升，改变梦境图像
  img = gradient_ascent(img,
                       iterations=iterations,
                       step=step,
                       max_loss=max_loss)
  
  #将原始图像的较小版本放大，它会变得像素化
  upscaled_shrunk_original_img = resize_img(shrunk_original_img,shape)
  
  #在这个尺寸上计算原始图像的高质量版本
  same_size_original = resize_img(original_img,shape)
  
  #二者的差别就是在放大过程中丢失的细节
  lost_detail = same_size_original - upscaled_shrunk_original_img
  
  #将丢失的细节重新注入到梦境图像中
  img += lost_detail
  shrunk_original_img = resize_img(original_img, shape)
  save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
  
save_img(img, fname='final_dream.png')
  

Processing image shape (199, 295)
...Loss value at 0 : 1.3767706
...Loss value at 1 : 1.7487947
...Loss value at 2 : 2.3076138
...Loss value at 3 : 3.016322
...Loss value at 4 : 3.7166796
...Loss value at 5 : 4.3557544
...Loss value at 6 : 5.0385723
...Loss value at 7 : 5.7042227
...Loss value at 8 : 6.2766848
...Loss value at 9 : 6.9151273
Processing image shape (259, 383)
...Loss value at 0 : 3.0095322
...Loss value at 1 : 4.3412023
...Loss value at 2 : 5.420557
...Loss value at 3 : 6.2674437
...Loss value at 4 : 7.0912175
...Loss value at 5 : 7.8543625
...Loss value at 6 : 8.565756
...Loss value at 7 : 9.199576
...Loss value at 8 : 9.871966
...Loss value at 9 : 10.49438
Processing image shape (337, 499)
...Loss value at 0 : 3.6534905
...Loss value at 1 : 5.217735
...Loss value at 2 : 6.3830504
...Loss value at 3 : 7.3438945
...Loss value at 4 : 8.250573
...Loss value at 5 : 9.034122
...Loss value at 6 : 9.774834
...Loss value at 7 : 10.507
...Loss value at 8 : 11.1452875
...Loss val